# 문화/게임 콘텐츠 용어 LLM 파인튜닝

## 모델 정보
- **Base Model**: `skt/kogpt2-base-v2`
- **Fine-tuning 방식**: LoRA
- **태스크**: 용어 정의 생성

---

## 1. 라이브러리 설치

In [ ]:
%%capture
!pip install transformers datasets accelerate peft bitsandbytes
!pip install sentencepiece

## 2. 환경 설정 및 라이브러리 임포트

In [ ]:
import torch
import json
from pathlib import Path
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType
from datasets import Dataset
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
DATA_PATH = Path('/content/drive/MyDrive/LLM_FineTuning_Project/processed/')
OUTPUT_PATH = Path('/content/drive/MyDrive/LLM_FineTuning_Project/model/')
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

print(f"데이터 경로: {DATA_PATH}")
print(f"모델 저장 경로: {OUTPUT_PATH}")

## 3. 모델 및 토크나이저 로드

In [ ]:
# 모델명
MODEL_NAME = "skt/kogpt2-base-v2"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    bos_token='</s>',
    eos_token='</s>',
    unk_token='<unk>',
    pad_token='<pad>',
    mask_token='<mask>'
)

print(f"토크나이저 로드 완료!")
print(f"어휘 크기: {len(tokenizer):,}")

In [ ]:
# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,
    device_map='auto',
    trust_remote_code=True
)
model.config.use_cache = False

print(f"모델 로드 완료!")
print(f"모델 파라미터 수: {model.num_parameters():,}")

## 4. LoRA 설정

In [ ]:
# 모델 구조 확인
print("모델 모듈 이름 확인:")
for name, module in model.named_modules():
    if 'attn' in name.lower() or 'proj' in name.lower():
        print(f"  {name}")

# LoRA 설정
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,                        # LoRA rank
    lora_alpha=32,              # LoRA alpha
    lora_dropout=0.1,           # Dropout
    target_modules=["c_attn"],  # KoGPT2의 attention 모듈
    inference_mode=False,       # 학습 모드
    bias="none",                # bias 학습 안함
)

model.enable_input_require_grads()

model = get_peft_model(model, lora_config)

for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"학습 가능: {name}")
        break  

model.print_trainable_parameters()

## 5. 데이터 로드 및 전처리

In [ ]:
def load_jsonl(file_path):
    """JSONL 파일 로드"""
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return data

# 데이터 로드
train_data = load_jsonl(DATA_PATH / 'train.jsonl')
val_data = load_jsonl(DATA_PATH / 'val.jsonl')

print(f"학습 데이터: {len(train_data):,}개")
print(f"검증 데이터: {len(val_data):,}개")

In [ ]:
# 프롬프트 템플릿
PROMPT_TEMPLATE = """### 질문:
{instruction}

### 답변:
{output}</s>"""

def format_prompt(example):
    """프롬프트 형식으로 변환"""
    return PROMPT_TEMPLATE.format(
        instruction=example['instruction'],
        output=example['output']
    )

# 예시 확인
print("=" * 50)
print("프롬프트 형식 예시:")
print("=" * 50)
print(format_prompt(train_data[0]))

In [ ]:
# 토큰화 함수
MAX_LENGTH = 512

def tokenize_function(examples):
    """텍스트를 토큰화"""
    texts = [format_prompt(ex) for ex in examples]

    tokenized = tokenizer(
        texts,
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length',
        return_tensors='pt'
    )

    # labels = input_ids (Causal LM)
    tokenized['labels'] = tokenized['input_ids'].clone()

    return tokenized

# Dataset 생성
train_texts = [format_prompt(ex) for ex in train_data]
val_texts = [format_prompt(ex) for ex in val_data]

train_dataset = Dataset.from_dict({'text': train_texts})
val_dataset = Dataset.from_dict({'text': val_texts})

print(f"Train Dataset: {len(train_dataset)}")
print(f"Validation Dataset: {len(val_dataset)}")

In [ ]:
# 토큰화 적용
def tokenize_dataset(examples):
    return tokenizer(
        examples['text'],
        truncation=True,
        max_length=MAX_LENGTH,
        padding='max_length'
    )

train_tokenized = train_dataset.map(
    tokenize_dataset,
    batched=True,
    remove_columns=['text']
)

val_tokenized = val_dataset.map(
    tokenize_dataset,
    batched=True,
    remove_columns=['text']
)

print("토큰화 완료!")
print(f"Train 샘플 키: {train_tokenized[0].keys()}")

## 6. 학습 설정

In [ ]:
# 학습 인자 설정
training_args = TrainingArguments(
    output_dir=str(OUTPUT_PATH / 'checkpoints'),

    # 학습 하이퍼파라미터
    num_train_epochs=3,
    per_device_train_batch_size=4, 
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=4, 

    # 학습률 설정
    learning_rate=2e-4,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,

    # 메모리 최적화
    fp16=True,
    gradient_checkpointing=True,

    # 로깅 및 저장
    logging_steps=100,
    eval_strategy='steps',
    eval_steps=500,
    save_strategy='steps',
    save_steps=500,
    save_total_limit=3,
    load_best_model_at_end=True,

    report_to='none',
    seed=42,
)

print("학습 설정 완료")
print(f"  - Epochs: {training_args.num_train_epochs}")
print(f"  - Batch Size: {training_args.per_device_train_batch_size}")
print(f"  - Gradient Accumulation: {training_args.gradient_accumulation_steps}")
print(f"  - Effective Batch Size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - Learning Rate: {training_args.learning_rate}")

In [ ]:
# Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  
)

# Trainer 생성
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=val_tokenized,
    data_collator=data_collator,
)

print("Trainer 생성 완료!")

## 7. 모델 학습

In [ ]:
# 학습 시작!
print("=" * 50)
print("학습 시작!")
print("=" * 50)

trainer.train()

In [ ]:
# 학습 결과 확인
print("\n" + "=" * 50)
print("학습 완료!")
print("=" * 50)

# 최종 평가
eval_results = trainer.evaluate()
print(f"\n검증 결과:")
for key, value in eval_results.items():
    print(f"  - {key}: {value:.4f}")

## 8. 모델 저장

In [ ]:
# LoRA 어댑터 저장
model.save_pretrained(OUTPUT_PATH / 'lora_adapter')
tokenizer.save_pretrained(OUTPUT_PATH / 'lora_adapter')

print(f"모델 저장 완료")
print(f"저장 위치: {OUTPUT_PATH / 'lora_adapter'}")

In [ ]:
# LoRA를 베이스 모델과 병합하여 전체 모델 저장
merged_model = model.merge_and_unload()
merged_model.save_pretrained(OUTPUT_PATH / 'merged_model')
tokenizer.save_pretrained(OUTPUT_PATH / 'merged_model')

print(f"병합 모델 저장 완료")
print(f"저장 위치: {OUTPUT_PATH / 'merged_model'}")